## Import y funciones auxiliares

In [15]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

# === helpers ===

def classify_age(age):
    age_cats = ['Adults (35-64)', 'Young Adults (25-34)', 'Youth (<25)']
    if age < 25:
        return age_cats[2]
    elif age < 35:
        return age_cats[1]
    else: 
        return age_cats[0]
    

## 1. Carga de datos

In [16]:
df_raw = pd.read_excel("./datasets/bikesales.xlsx")
df_clean = df_raw.copy()

## 2. Reglas de limpieza

### 2.1 Transformamos a tipos modernos

Acción: Transformar tipos del dataframe con df.convert_dtypes()

### 2.2 Normalización de nombres de columnas (Log 3.1)

Acción:
Normalizar nombres de columnas a estilo snake_case.

### 2.3 Columnas sin valor analitico (Log 3.2)

Acción:
Eliminar columnas **day**, **month**, **year**, **product_category** y **sub_category**

### 2.4 Extraer atributos de product_description

Acción: 
La columna **product_description** contiene atributos de talla y color que conviene separar en **frame_size** y **product_color** para labores de business analytics

### 2.5 Valores nulos (log 3.3)

##### Nulos en age_group

Acción:
Reconstruir a partir de **customer_age**

##### Nulos en order_quantity

Acción: 
Reconstruir a partir de **unit_cost** y **cost**

##### Nulos en product_description

Acción: 
Reconstruir a partir de **unit_cost**

### 2.6 Problemas categóricos (log 3.5)

Acción: 
Normalizar valores categoricos aplicando str.strip(), str.lower(), colapso de espacios y elminización de caracteres no estándar

### 2.7 Valores duplicados (log 3.4)

#### filas duplicadas

Acción: Tras normalizar valores aplicar df.drop_duplicates()

#### sales_order presenta un caso en el cual atribuye el mismo identificador a dos filas distintas.

Acción: se identificará el caso a través de una columna booleana **order_conflict**

### 2.8 valores imposibles (log 3.6)

#### Registros con cost = 0 y unit_cost = 0

Acción: inferir el valor correcto como:

 **cost** = **revenue** - **profit** 

 **unit_cost** = **cost** /  **order quantity**

#### Registros con revenue = 0 y unit_price = 0

Acción: inferir el valor correcto como:

 **revenue** = **cost** + **profit** 

 **unit_price** = **revenue** / **order_quantity**

## 3. Limpieza de datos

### Regla 1 - Transformación a tipos modernos

In [17]:
df_raw = df_raw.convert_dtypes()
df_clean = df_clean.convert_dtypes()

### Regla 2 - Limpieza de nombre de las columnas

In [18]:
df_clean.columns = (
    df_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ","_")
    .str.replace(r"[^0-9a-zA-Z_]+","", regex = True)
    .str.strip("_")
)

df_raw.columns = (
    df_raw.columns
    .str.strip()
    .str.lower()
    .str.replace(" ","_")
    .str.replace(r"[^0-9a-zA-Z_]+","", regex = True)
    .str.strip("_")
)

### Regla 3 - Eliminación columnas innecesarias

In [19]:
df_clean.drop("day", axis = 1, inplace = True)
df_clean.drop("month", axis = 1, inplace = True)
df_clean.drop("year", axis = 1, inplace = True)
df_clean.drop("product_category", axis = 1, inplace = True)
df_clean.drop("sub_category", axis = 1, inplace = True)

### Regla 4 - Derivación de columnas fram_size y product_color a partir de product_description

In [20]:
df_clean["frame_size"] = (
    df_raw.product_description
    .str.extract(r",\s*(\d+)$")
    .astype("Int64")
)

df_clean["product_color"] = (
    df_raw.product_description
    .str.extract(r"\s*([a-zA-Z]+),")
    .astype("string")
)

df_clean["product_description"] = (
    df_clean.product_description
    .str.replace(r",\s*\d+$","", regex = True)
    .str.replace(r"\s*[a-zA-Z]+$","", regex = True)
)


### Regla 5 - Reconstruir valores nulos

#### age_group a partir de customer_age

In [21]:
mask = df_clean["age_group"].isna()

df_clean.loc[mask, "age_group"] = (
    df_clean.loc[mask, "customer_age"]
    .apply(classify_age)
    .astype("string")
)

#### order_quantity a partir de unit_cost y cost

In [22]:
mask = df_clean["order_quantity"].isna()

df_clean.loc[mask, "order_quantity"] = (
    df_clean.loc[mask, "cost"] / df_clean.loc[mask, "unit_cost"]
)

#### product_description a partir de unit_cost

In [23]:
ref = (
    df_clean
    .loc[df_clean["product_description"].notna(), ["unit_cost", "product_description"]]
    .drop_duplicates()
    .groupby("unit_cost")["product_description"]
    .agg(["nunique", "first"])
)

valid_map = ref.loc[ref["nunique"] == 1, "first"].to_dict()

mask = df_clean["product_description"].isna()

df_clean.loc[mask, "product_description"] = (
    df_clean.loc[mask, "unit_cost"].map(valid_map)
)



### Regla 6 - Normalizar valores categoricos

In [24]:
cat_cols = df_clean.select_dtypes(include=["string"]).columns

for col in cat_cols: 
    df_clean[col] = (
        df_clean[col]
        .astype("string")
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True) 
    )

### Regla 7 - Valores duplicados

In [25]:
# Eliminamos filas duplicadas
df_clean = df_clean.drop_duplicates().reset_index(drop=True)
df_clean.head(10)

,sales_order,date,customer_age,age_group,customer_gender,country,state,product_description,order_quantity,unit_cost,unit_price,profit,cost,revenue,frame_size,product_color
0,261695,2021-12-01,39,adults (35-64),f,united states,california,mountain-200,4,1252,2295,4172,5008,9180,46,black
1,261695,2021-12-01,44,adults (35-64),m,united kingdom,england,mountain-200,1,1266,2320,1054,1266,2320,42,silver
2,261697,2021-12-02,37,adults (35-64),m,united states,california,mountain-400-w,2,420,769,698,840,1538,46,silver
3,261698,2021-12-02,31,young adults (25-34),f,australia,new south wales,mountain-400-w,1,420,769,349,420,769,42,silver
4,261699,2021-12-03,37,adults (35-64),f,united states,california,mountain-200,2,0,2295,2086,0,4590,46,black
5,261700,2021-12-03,24,youth (<25),f,united kingdom,england,mountain-200,1,1252,2295,1043,1252,2295,38,black
6,261701,2021-12-03,37,adults (35-64),m,united states,washington,mountain-200,1,1252,2295,1043,1252,2295,46,black
7,261702,2021-12-04,31,young adults (25-34),f,australia,new south wales,mountain-400-w,4,420,0,1396,1680,0,42,silver
8,261703,2021-12-05,39,adults (35-64),f,united states,california,mountain-200,4,1252,2295,4172,5008,9180,46,black
9,261704,2021-12-05,42,adults (35-64),m,germany,nordrhein-westfalen,mountain-200,4,1252,2295,4172,5008,9180,38,black


In [26]:
# Creamos columna booleana para identificar orders conflictivas
conflict_orders = (
    df_clean.groupby("sales_order")["customer_age"]
            .nunique()
            .loc[lambda s: s > 1]
            .index
)

df_clean["order_conflict"] = df_clean["sales_order"].isin(conflict_orders).astype("boolean")

### Regla 8 - Reconstrucción valores imposibles

#### Registro cost = 0 

In [27]:
mask = df_clean["cost"].eq(0)

df_clean.loc[mask, "cost"] = (
    df_clean.loc[mask, "revenue"] - df_clean.loc[mask, "profit"]
)

#### Registro unit_cost = 0 

In [28]:
mask = df_clean["unit_cost"].eq(0)

df_clean.loc[mask, "unit_cost"] = (
    df_clean.loc[mask, "cost"] / df_clean.loc[mask, "order_quantity"]
)

#### Registro revenue = 0


In [29]:
mask = df_clean["revenue"].eq(0)

df_clean.loc[mask, "revenue"] = (
    df_clean.loc[mask, "cost"] + df_clean.loc[mask, "profit"]
)

#### Registro unit_price = 0

In [30]:
mask = df_clean["unit_price"].eq(0)

df_clean.loc[mask, "unit_price"] = (
    df_clean.loc[mask, "revenue"] / df_clean.loc[mask, "order_quantity"]
)

### Reordenamos columnas para mejorar legibilidad

In [31]:
cols = [
        'sales_order',
        'order_conflict',
        'date',
        'customer_age',
        'age_group',
        'customer_gender',
        'country',
        'state',
        'product_description',
        'product_color',
        'frame_size',
        'order_quantity',
        'unit_cost',
        'unit_price',
        'profit',
        'cost',
        'revenue'
]
df_clean = df_clean[cols]

### Exportamos excel limpio

In [32]:
df_clean.to_excel("./datasets/bikesales_clean.xlsx", index = False)